<a href="https://colab.research.google.com/github/deepakdm04/Kaiburr-Task-5---ML/blob/main/Task5Ml-Kaiburr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install necessary packages (if needed)
!pip install nltk scikit-learn matplotlib seaborn

# Import libraries
import pandas as pd
import numpy as np
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd

file_path = '/content/drive/MyDrive/complaints.csv/complaints.csv'


df = pd.read_csv(file_path)
df.shape, df.columns


<ipython-input-5-1915338173>:7: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


((9420854, 18),
 Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
        'Consumer complaint narrative', 'Company public response', 'Company',
        'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
        'Submitted via', 'Date sent to company', 'Company response to consumer',
        'Timely response?', 'Consumer disputed?', 'Complaint ID'],
       dtype='object'))

In [8]:
category_map = {
    'Credit reporting, credit repair services, or other personal consumer reports': 0,
    'Debt collection': 1,
    'Consumer Loan': 2,
    'Mortgage': 3
}

df = df[df['Product'].isin(category_map.keys())]
df = df[df['Consumer complaint narrative'].notna()]
df['Category'] = df['Product'].map(category_map)
df[['Product', 'Consumer complaint narrative', 'Category']].head()


,Product,Consumer complaint narrative,Category
197,Mortgage,I purchased a home from Illinois to move to Lo...,3
237,Debt collection,I've complained about procollect and this fals...,1
240,Debt collection,Credit Collections obtained access to my consu...,1
1844,Debt collection,"First of all, I never received any email or of...",1
3124,Debt collection,I was not served to appear in court for either...,1


In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower().split()
    text = [stemmer.stem(word) for word in text if word not in stop_words]
    return ' '.join(text)

df['clean_text'] = df['Consumer complaint narrative'].apply(preprocess)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['clean_text']).toarray()
y = df['category']

X.shape, y.shape


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "SVM": LinearSVC()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"{name}: Accuracy = {acc:.4f}")


In [ ]:
best_model = LogisticRegression(max_iter=1000)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
sample_text = ["The mortgage company keeps calling me even after full payment."]
processed = tfidf.transform([preprocess(sample_text[0])]).toarray()
predicted_class = best_model.predict(processed)[0]

category_name = [k for k, v in category_map.items() if v == predicted_class][0]
print("Predicted Category:", category_name)
